# ¿Cómo probamos un modelo?
La mejor manera de probar un modelo es utilizarlo el mundo real, mandarlo a enfrentar la vida prediciendo nuevos datos que no haya aprendido.

¿Pero cómo sabemos si está listo?

Reservaremos un conjunto que no verá durante el entrenamiento, el **test set** o conjunto de prueba. Por supuesto, el otro conjunto será el **training set** o conjunto de entrenamiento.

![training_test_set](https://data-flair.training/blogs/wp-content/uploads/sites/2/2018/08/1-16.png)

Con frecuencia, se divide el dataset en 80% para training y 20% para test. Valores de 70-30 o 60-40 también son comunes.  
Pero **antes debemos mezclarlo** como una baraja de naipes, ¿por qué? necesitamos una muestra aleatoria que represente a todo el conjunto.

Imaginemos que los datos fueron tomados por zonas y en orden, una parte, en el orden original, podrían ser las viviendas de una zona específica, esa parte podría faltar en el entrenamiento y nuestro modelo no aprenderá de ella.

Si tomamos datos aleatorios, tendremos una muestra que represente en **general** a todo el dataset.

## Primero mezclamos
Sea n el tamaño del dataset.  
Definiremos una [semilla](https://es.wikipedia.org/wiki/Semilla_aleatoria). que los resultados sean [reproducibles](https://es.wikipedia.org/wiki/Reproducibilidad_y_repetibilidad), esto es, siempre los mismos.  
Creamos un array de 0 a n-1 que representará los **indices**, lo desordenamos al azar y reemplazamos **X** e **y** con los **indices desordenados**, es importante que sean los **mismos indices**, cada **X** tiene su correspondiente **y**.

In [ ]:
import numpy as np
from sklearn.datasets import load_boston

# semilla
np.random.seed(42)

#probemos usar
X, y = load_boston(return_X_y=True)

# ¿cuántos datos tenemos?
n = len(X)
n

In [ ]:
# array de 0 a n-1
indices = np.arange(n)

print(indices[:5], "...", indices[-3:])

In [ ]:
# desordenamos
np.random.shuffle(indices)

print(indices[:5], "...")

# reemplazamos
X = X[indices]
y = y[indices]

## Ahora dividimos
Antes debemos seleccionar usar alguna de [sus features](https://scikit-learn.org/stable/datasets/index.html#boston-house-prices-dataset), probemos con la última.  
Dividamos en 60-40, necesitamos el indice que marque dónde termina el 60%  
Tanto **train set** como **test set** son términos que agrupan ambas variables X e y.

In [ ]:
# todas las filas, última columna
X_selected = X[:, -1]

# el índice debe ser entero
training_size = int(n * 0.6)

# seleccionamos el primer 60%
X_train = X_selected[:training_size]
y_train = y[:training_size]

# y el l 40% restante
X_test = X_selected[training_size:]
y_test = y[training_size:]

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(X_train, y_train)
plt.scatter(X_test, y_test)
plt.xlabel("LSTAT")
plt.ylabel("Valor")
plt.title("Conjuntos de entrenamiento y prueba")

Ambos conjuntos son representativos, ¿verdad?

## Probando, probando
Primero entrenamos **sólo en el train set**.  

Luego predecimos y calculamos el **MAE** usando los datos no vistos, el **test set**.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(X_train.reshape(-1, 1), y_train)

y_hat = model.predict(X_test.reshape(-1, 1))
print(f"MAE (test set) {mean_absolute_error(y_hat, y_test) :.2f}")

Esta cantidad es más fiable, como el modelo no conoce estos datos, que además son reales, podemos decir que  nuestro modelo ha enfrentado la vida real :D

Pero mientras mejoramos el modelo calculamos el error del train set, anímate a **c_mpletar el código**:

In [ ]:
# obtenemos los valores predecidos usando el train set
y_hat_train = model.___(X___.reshape(-1, 1))

# calcularmos el MAE del train set
print(f"MAE (train set): {___error(___, y_tr___) :.2f}")

La mayoría de las veces, el error del train set será menor, porque son los datos que "mejor conoce" el modelo, píensa que mientras más sepas de algo, mejor habrás aprendido.

# Regresión polinomial
Mira la gráfica, notarás que una recta no es suficiente para ajustar bien los datos, hace falta una **curva** cuya ecuación es un **polinomio**:

$$P(x) = a_nx^n + a_{n-1}x^{n-1} + \cdots + a_0x^0$$

Notarás que se parece a la regresión lineal múltiple, con una pequeña gran diferencia, se usa la **misma feature elevada a un exponente**, esto se conoce como regresión **polinomial**, y se ajusta como si fuera una regresión lineal múltiple.

Probaremos elevando a dos, incluyendo la misma feature al cuadrado el modelo pensará que son features diferentes :D  
Podemos "apilar" varios arrays contenidos en una tupla utilizando `column_stack` de numpy, esto tratará cada array como una columna y es necesario que todos los arrays tengan la misma longitud.

In [ ]:
X_poly_train = np.column_stack((X_train ** 2, X_train))
X_poly_train[:5]

Entrenando el modelo, de seguro tendremos un error menor

In [ ]:
model_poly = LinearRegression()
model_poly.fit(X_poly_train, y_train)

y_hat_poly = model_poly.predict(X_poly_train)
print(f"MAE: {mean_absolute_error(y_hat_poly, y_train) :.2f}")

## Veamos la curva
Si unimos sólo 2 puntos tendremos una recta aunque usemos la ecuación polinómica de `model_poly`.

In [ ]:
xpoints = np.array([min(X_train), max(X_train)])
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))

ypoints = model_poly.predict(xpoints_poly)

plt.plot(xpoints, ypoints, color="red")

Necesitamos valores intermedios, mientras más tengamos, más suave será nuestra curva. La función `linspace` de numpy nos permite generar n valores equidistantes entre sí en un rango definido.

Generemos 5 valores, el rango será el mínimo y el máximo de X_train, que contiene los valores originales, sin elevar al cuadrado.

In [ ]:
xpoints = np.linspace(start=min(X_train), stop=max(X_train), num=5)
xpoints

In [ ]:
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))
ypoints = model_poly.predict(xpoints_poly)

plt.plot(xpoints, ypoints, color="red")

Para suavizar la curva no es necesario generar **demasiados** valores.
Veamos también el train set.

In [ ]:
xpoints = np.linspace(start=min(X_train), stop=max(X_train), num=15)
xpoints_poly = np.column_stack((xpoints ** 2, xpoints))
ypoints = model_poly.predict(xpoints_poly)

plt.plot(xpoints, ypoints, color="red")
plt.scatter(X_train, y_train)

## Genera polinomios
El objeto [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn-preprocessing-polynomialfeatures) de sklearn convierte las features en polinomios, el parámetro `degree` indica el [grado](https://es.wikipedia.org/wiki/Grado_(polinomio)), esto es, el exponente máximo al que elevaremos.

Su método `fit_transform` nos devolverá una matriz con cada fila en este formato: $x^0, x^1, \cdots, x^n$ si el parámetro es **una** sola feature. En realidad este método llama a otros dos `fit` para calcular qué features tendremos y `transform` para crear dichas featuras.

Como $x^0 = 1$ tendremos una columna llena de unos, este término se conoce como **bias** o sesgo, por ahora lo excluiremos estableciendo `include_bias` en falso.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_train.reshape(-1, 1))
X_poly[:5]

Volvemos a necesitar el reshape a (-1, 1) porque podemos incluir varias features, en caso de hacerlo tendremos un **polinomio de varias variales**.  
Veamos uno de dos variables y grado dos sin coeficientes:

${x_1}^2 + x_1 + x_1x_2 + x_2 +{x_2}^2$

El exponente máximo es dos, aunque el término del medio tenga las dos variables, la suma de sus exponentes resulta en 2 pues ambos tienen grado 1.  
De modo que un término con la forma ${x_1}^2x_2$ tendría grado 3.

En nuestro caso de dos variables y grado 2 `PolynomialFeatures` devolverá una lista con el formato $x_1, x_2, {x_1}^2, x_1x_2, {x_2}^2$

In [ ]:
# cambiamos de grado
poly = PolynomialFeatures(degree=2, include_bias=False)

# tomamos dos features
X_selected = X[:, [5, 7]]

# recuerda que sólo tomamos una parte del dataset
X_train = X_selected[:training_size]

X_poly_train = poly.fit_transform(X_train)
X_poly_train[:5]

## ¡Pero no generes tantos!
Prueba aumentar el grado, verás más columnas, y muchas más si añades features antes de generar polinomios.

Como mencioné, estas columnas cuentan como nuevas features para el modelo, nuevas features pueden disminuir el error, pero por cada columna **añades una dimensión** y esto causa un problema conocido como **curse of dimensionality** o maldición de la dimensión.

Esto trae "problemas que no se ven en menores dimensiones", como la **dispesión** y la **cercanía** de los datos.  
Lo explicaré con un juego, estás en el principio y debes debes alcanzar un objeto que está al final en un espacio de 3 secciones:

    OX*
    
Sólo debes recorrer 2 secciones para alcanzarlo, estás muy cerca del objeto, lo alcanzas y pasas de nivel.

Ahora el juego tiene de un espacio de 3 secciones de largo y 3 de alto, siendo 9 en total:

    XX*
    XXX
    OXX

Sólo debes recorrer 4 secciones para alcanzar el objeto, ya no está tan cerca pero no tardas en alcanzarlo y pasar a otro nivel.

Se añade otra dimensión también con 3 secciones, tendemos un juego 3D con 27 secciones y la misma situación.
¿Cuántas secciones debes recorrer? 

Al añadir dimensiones, tú y el objeto se han **dispersado** perdiendo **cercanía**, lo mismo pasa con los datos cuando añadimos features, será más dificil para el modelo encontrar la recta que mejor se ajuste porque tenemos distancias más grandes.

A menos que la dimensión extra nos brinde mejor información o tengamos un modelo más complejo, como una curva que pueda acercarse a los datos.  
Esta excepción es importante, un modelo **más complejo** sólo será recomendable si tiene **más datos** de los que aprender.

# Probando la curva
Veamos si el modelo polinomial se desempeña mejor al final, para esto es necesario transformar también el test set, usaremos `transform` en este ya que tendrá las mismas features y no necesitamos volver a calcularlas.

Siendo la sección final, **debes completarla** con lo visto en este capítulo.

In [ ]:
# nuestra feature original
X_selected = X[:, -1]

# toma ambos conjuntos
X_train = X_selected[:tr___siz_]
X_test = X_selected[___:]

# ¡transforma!
poly = P___ialFeatures(degree=2, include_bias=___)

X_poly_train = poly.fit_tr___(X_tr___.reshape(___))
X_poly_test = poly.transform(X_tes___.r___(___))

# entrena
model_poly = ___Regr_ssion()
model_poly.___(X_poly_train, y_tr___)

# predice
y_hat_train = model_poly.pr___(X_poly_train)
y_hat_test = model_poly.___(X_poly_test)

# calcula los errores para comparar
print(f"MAE (test set): {___error(y_h___, y_te__) :.2f}")
print(f"MAE (training set): {___err___(y___, y___) :.2f}")

Tenemos una diferencia considerable entre ambos conjuntos, pero si ambos son representativos y ambos son una curva ¿qué pasó entonces? 

Un modelo más complejo permite aprender mejor, si aprendes mejor unas cuantas cosas... estarías perdiendo la idea general.  
Lo mismo pasa con machine learning, es importante que lo aprendido se pueda **generalizar**, que funcione fuera de lo aprendido, de lo contrario tenemos un problema llamado **overfitting** o [sobreajuste](./4_ajuste).